In [ ]:
## IMPORTANT: On Colab, we expect your homework to be in the cs189 folder
## Please contact staff if you encounter any problems with installing dependencies
import sys
IS_COLAB = 'google.colab' in sys.modules
if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    %cd /content/drive/MyDrive/cs189/hw/hw3
    %pip install -r ./requirements.txt
    !pip install -U kaleido plotly
    import kaleido
    kaleido.get_chrome_sync()

import plotly.io as pio
pio.renderers.default = pio.renderers.default + "+png"


In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hw3.ipynb")

<link rel="stylesheet" href="berkeley.css">

<h1 class="cal cal-h1">Homework 03 – Optimizers and Backpropagation</h1>

CS 189, Fall 2025

Welcome to Homework 3! In this assignment, you will use your newfound skills in backpropagation to implement an autodifferentiation library from scratch. You'll extend the single-variable autograd that we learned in lecture to function with general tensors, mimicking how `torch`'s autograd is implemented in research and industry.

---

## Due Date: November 7th, 2025

This assignment is due on **November 7th, 2025**. You must submit your work to Gradescope by this deadline. Please refer to the syllabus for the [Slip Day policy](https://eecs189.org/fa25/syllabus/#slip-days). No late submissions will be accepted beyond the details outlined in the Slip Day policy.

### Submission Tips
- **Plan ahead**: We strongly encourage you to submit your work several hours before the deadline. This will give you ample time to address any submission issues.
- **Reach out for help early**: If you encounter difficulties, contact course staff well before the deadline. While we are happy to assist with submission issues, we cannot guarantee responses to last-minute requests.

<!-- ---

### Assignment Overview

This notebook contains a series of tasks designed to help you practice and apply theoretical concepts you learned in class.

Question 1: Implementing Backpropagation

Question 2 : Implementing Optimizers

--- -->

### Key Learning Objectives

1. Learn how backpropagation is implemented in libraries like PyTorch
2. Understand how optimizers are implemented, and the advantages of Adam
    
---

### Collaboration Policy
You are encouraged to discuss high-level concepts with your peers. However:
- All submitted work must be written in your own words and code.
- Do not share or copy solutions directly.
- List any collaborators (students you worked with) in the line below. Include their name and SID:

**Your Collaborators**: **TODO**

### AI Tools Usage Disclosure
We allow the use of AI tools (e.g., ChatGPT, Copilot) **only as support**, not as a replacement for your own reasoning. To ensure transparency, you must acknowledge any use of AI tools.

Please answer with one of the following options:
- **A) I did not use any AI tools for this homework.**
- **B) I used AI tools in the following way(s):**  
  (describe briefly, e.g., “Used ChatGPT to get hints for debugging a NumPy indexing error”)


**Your Answer**: **TODO**
    
---

### Grading Breakdown

<!-- <div align="center"> -->


| Question  | Manual Grading? | Points |
| --------- | --------------- | ------ |
| q1        | No              | 10     |
| q2        | No              | 20     |
| q3        | No              | 10     |
| q4        | No              | 10     |
| **Total** |                 | **50** |


In [ ]:
from __future__ import annotations
import numpy as np
from dataclasses import dataclass
import matplotlib.pyplot as plt
from collections import deque
import typst
import re
np.random.seed(189)

**Introducing BearTensor** 

Before we can dig into our multivariable calculus, we first need a good abstraction for tensors,
which are just $n$-dimensional arrays (similar to `numpy`'s `ndarray`). `numpy` doesn't
provide autodifferentiation for its `ndarray`, so we can build a wrapper around it. For this homework assignment we are going to be using the `BearTensor` class, which you can think of as a custom implementation of `torch.Tensor` that acts as a wrapper around `numpy` arrays.

Take a look at the `BearTensor`, `BearGrad`, and `BearParent` classes below.

**Question 1**: Building the Computation Graph

**Conceptual Overview**:

Our model needs some way to build a computation graph for backpropagation so that it knows where to pass down gradients during the backwards pass. The way that PyTorch does this, is that every time you combine two values to compute a third one in the forward pass, it will keep track of this internally. We want to implement a similar functionality for `BearTensor`; everytime we add, subtract, divide, etc. two `BearTensor` to get a third one, we should keep track of the fact that the third tensor's **parents** are the two source tensors. 

Here is an example of what a computation graph might look like from lecture:
<div style="text-align: center;">
  <img src="https://imgur.com/PO3S6TI.png" alt="Backpropagation" style="display: block; margin-left: auto; margin-right: auto; width: 60%;">
</div>

**Your Task**:

Currently, our `BearTensor` has no operations. Implement the following functions in the class:
- Addition (`__add__`)
- Subtraction (`__sub__`)
- Multiplication (`__mul__`)
- Power (`__pow__`)
- Matrix multiplication (`__matmul__`)
- Dot product (`dot`)
- Sum (`sum`)
- Mean (`mean`)
- ReLU (`relu`)
- Sigmoid (`sigmoid`)

A few things to remember:
1. You should return a new `BearTensor` from these functions without modifying the existing one
2. Remember to set the parents of this new Tensor to be the two source tensors; this is how we will build a computation graph
3. Remember to set the correct gradient function as the first argument of `BearParent.BearGrad`; this will tell us how to compute the downstream gradient during the backward pass

Additionally, we are going to be doing a lot NumPy operations in this Homework. It is important that you think about how the shapes of different arrays will change due to NumPy broadcasting rules, and reshape when needed. Some examples of this that are relevant to this HW are:
- NumPy will automatically broadcast 1-element arrays to a scalar (ex. when computing the dot product of two vectors)
- NumPy will broadcast array of shape (N, 1) to just (N,) (ex. when a matrix times a matrix gives a vector)

**Note**: The public tests that we've provided in this notebook are NOT fully-comprehensive and often very simple. You should build more complex graphs and manually verify that your computed gradients and values are correct. Our hidden tests are more comprehensive and will check for full correctness 


In [ ]:
@dataclass
class BearGrad:
    '''Stores how to compute the downstream gradient from the upstream gradient 
    - `op_str` is just a string describing the operation; you don't need to use this for this HW, but it may help in debugging if you print out what operations create your computation graph.
    - `fn` is a function that takes in the upstream loss gradient and outputs the loss gradient that should be passed downstream. This essentially applies the chain rule at the current node in the computation graph.
    '''
    fn: callable
    op_str: str | None = None

@dataclass
class BearParent:
    '''This class represents a parent of a node; a node must track its parents so that it knows who to propagate its gradients to.
    - `grad` is the `BearGrad` object for this parent; we can apply the `fn` from this to compute the gradient that should be passed downstream.
    - `parent` is the `BearTensor` object for the parent
    '''
    parent: BearTensor
    grad: BearGrad

    def __hash__(self):
        return id(self)

    def __eq__(self, other):
        return self is other

class BearTensor:
    '''`BearTensor`: This represents a node in our computation graph
    - `value` is the underlying data in the Tensor; this is computed during the forward pass
    - `parents` keeps track of the parents in the computation graph to whom we pass our gradient to
    - `adjoint` is the gradient (the transpose of it technically) that we compute in the backward pass
    '''
    def __init__(self, name: str, value: np.ndarray, parents: list[BearParent] | None = None):
        self.name = name
        self.value = value
        self.parents = parents if parents is not None else []
        self.adjoint: float | np.ndarray = 0.0

    def __add__(self, other: BearTensor) -> BearTensor:
        if self.value.shape != other.value.shape:
            raise ValueError("Shapes must match")
        ...

    def __sub__(self, other: BearTensor) -> BearTensor:
        if self.value.shape != other.value.shape:
            raise ValueError("Shapes must match")
        ...

    def __mul__(self, other: BearTensor) -> BearTensor:
        if self.value.shape != other.value.shape:
            raise ValueError("Shapes must match")
        ...

    def __pow__(self, power: float) -> BearTensor:
        ...

    def __matmul__(self, other: BearTensor) -> BearTensor:
        # This may be helpful to avoid shape mismatch errors
        def ensure_2d(x):
            '''If x is a scalar or 1-D, convert to 2-D'''
            x = np.asarray(x)
            if x.ndim == 0:          # scalar
                return x.reshape(1, 1)
            elif x.ndim == 1:        # vector
                return x.reshape(-1, 1)  # column vector convention
            else:                     # already 2D
                return x
        ...

    def dot(self, other: BearTensor) -> BearTensor:
        if self.value.ndim != 1 or other.value.ndim != 1:
            raise ValueError("dot() only supports 1-D BearTensors (like torch.dot).")
        ...

    def sum(self) -> BearTensor:
        ...

    def mean(self) -> BearTensor:
        ...

    
    def relu(self) -> BearTensor:
        ...

    def sigmoid(self) -> BearTensor:
        ...

**Disclaimer**: The Q1 test cases will NOT test that your adjoint calculations are correct until you implement Q2; if you are struggling with the next question, we recommend also looking back at your solution to Q1.

In [ ]:
grader.check("q1")

Below, we have provided some code that will allow you to visualize your computation graphs and an example for how to use it. You are not required to use nor understand how this works, but it may help you debug while working on this homework.

In [ ]:
def to_typst_math(s: str) -> str:
    processed_s = s
    processed_s = processed_s.replace(" @ ", " ")
    processed_s = re.sub(r"([a-zA-Z])([0-9]+)", r"\1_\2", processed_s)
    processed_s = re.sub(r"\.([a-zA-Z]+)", r'."\1"', processed_s)
    processed_s = re.sub(r"([a-zA-Z]+)_([a-zA-Z_]+)", r'\1_"\2"', processed_s)
    processed_s = re.sub(r"\b(relu|sum)\b", r'"\1"', processed_s)
    return processed_s


def _traverse_graph(
    root: BearTensor,
) -> tuple[list[BearTensor], list[tuple[BearTensor, BearTensor, str]]]:
    nodes, edges = set(), set()
    visited = set()

    def build(v):
        if v not in visited:
            visited.add(v)
            nodes.add(v)
            for p in v.parents:
                edges.add((p.parent, v, p.grad.op_str))
                build(p.parent)

    build(root)
    return list(nodes), list(edges)


def _minimize_crossings(
    nodes_by_level: list[list[BearTensor]],
    edges: list[tuple[BearTensor, BearTensor, str]],
    iterations: int = 4,
) -> list[list[BearTensor]]:
    node_to_parents = {n: [] for level in nodes_by_level for n in level}
    node_to_children = {n: [] for level in nodes_by_level for n in level}
    for p, c, _ in edges:
        if p in node_to_children and c in node_to_parents:
            node_to_children[p].append(c)
            node_to_parents[c].append(p)

    ordered_levels = [list(level) for level in nodes_by_level]

    for _ in range(iterations):
        for level_idx in range(1, len(ordered_levels)):
            child_level = ordered_levels[level_idx - 1]
            child_ranks = {node: rank for rank, node in enumerate(child_level)}

            def barycenter_down(node):
                children = node_to_children.get(node, [])
                if not children:
                    return -1
                avg_rank = sum(child_ranks.get(c, 0) for c in children) / len(children)
                return avg_rank

            ordered_levels[level_idx].sort(key=barycenter_down)

        for level_idx in range(len(ordered_levels) - 2, -1, -1):
            parent_level = ordered_levels[level_idx + 1]
            parent_ranks = {node: rank for rank, node in enumerate(parent_level)}

            def barycenter_up(node):
                parents = node_to_parents.get(node, [])
                if not parents:
                    return -1
                avg_rank = sum(parent_ranks.get(p, 0) for p in parents) / len(parents)
                return avg_rank

            ordered_levels[level_idx].sort(key=barycenter_up)

    return ordered_levels


def _assign_node_levels(
    root: BearTensor,
    nodes: list[BearTensor],
    edges: list[tuple[BearTensor, BearTensor, str]],
) -> dict[BearTensor, tuple[float, float]]:
    node_to_id = {node: i for i, node in enumerate(nodes)}
    id_to_node = {i: node for i, node in enumerate(nodes)}

    adj = {node_to_id[n]: [] for n in nodes}
    for p, c, _ in edges:
        if c in node_to_id and p in node_to_id:
            adj[node_to_id[c]].append(node_to_id[p])

    levels = {n: -1 for n in nodes}
    if root in levels:
        levels[root] = 0

    q = deque([root])
    max_level = 0
    visited_bfs = {root}

    while q:
        u = q.popleft()
        u_id = node_to_id[u]
        for v_id in adj.get(u_id, []):
            v = id_to_node.get(v_id)
            if v and v not in visited_bfs:
                levels[v] = levels[u] + 1
                max_level = max(max_level, levels[v])
                visited_bfs.add(v)
                q.append(v)

    nodes_by_level = [[] for _ in range(max_level + 1)]
    for node, level in levels.items():
        if level != -1:
            nodes_by_level[level].append(node)

    for level_nodes in nodes_by_level:
        level_nodes.sort(key=lambda n: n.name)

    nodes_by_level = _minimize_crossings(nodes_by_level, edges)

    node_coords = {}
    for level, nodes_in_level in enumerate(nodes_by_level):
        num_in_level = len(nodes_in_level)
        for rank, node in enumerate(nodes_in_level):
            y_pos = rank - (num_in_level - 1) / 2.0
            x_pos = -level
            node_coords[node] = (x_pos, y_pos)

    return node_coords


def _generate_typst_source(
    nodes: list[BearTensor],
    edges: list[tuple[BearTensor, BearTensor, str]],
    coords: dict[BearTensor, tuple[float, float]],
) -> str:
    node_to_id = {node: i for i, node in enumerate(nodes)}

    header = """#import "@preview/fletcher:0.5.8": diagram, node, edge
#set page(width: auto, height: auto, margin: 10mm, fill: white)
#set text(font: "Linux Libertine", size: 10pt)

#diagram(cell-size: (40mm, 40mm), {
"""

    node_definitions = ""
    for n in nodes:
        if n not in coords:
            continue
        x, y = coords[n]
        node_id = node_to_id[n]
        math_name = to_typst_math(n.name)
        shape_str = str(n.value.shape)
        content = f"""block(stroke: 0.5pt, inset: 8pt, radius: 4pt, [\n#align(center, ${math_name}$)\n#line(length: 100%)\n#text(size: 9pt, `shape: {shape_str}`)\n])"""
        node_definitions += f'  node(({x}, {y}), {content}, name: "{node_id}")\n'

    edge_definitions = ""
    LABEL_LENGTH_THRESHOLD = 20

    for parent, child, op_str in edges:
        from_id = node_to_id.get(parent)
        to_id = node_to_id.get(child)
        if from_id is None or to_id is None:
            continue

        op_str = op_str or ""
        math_op_str = to_typst_math(op_str)
        label = f"${math_op_str}$"

        if len(op_str) > LABEL_LENGTH_THRESHOLD:
            edge_definitions += f'  edge(label("{from_id}"), label("{to_id}"), "->", label: {label}, label-sep: 4em)\n'
        else:
            edge_definitions += (
                f'  edge(label("{from_id}"), label("{to_id}"), "->", label: {label})\n'
            )

    footer = "})"

    return header + node_definitions + edge_definitions + footer


def draw_graph(root: BearTensor, output_typ_path: str = "computational_graph.typ"):
    nodes, edges = _traverse_graph(root)
    coords = _assign_node_levels(root, nodes, edges)
    typst_source = _generate_typst_source(nodes, edges, coords)

    with open(output_typ_path, "w", encoding="utf-8") as f:
        f.write(typst_source)
    typst.compile(output_typ_path, output=output_typ_path.replace(".typ", ".pdf"))

# Example code
a = BearTensor("a", np.array([2, 3]))
b = BearTensor("b", np.array([1, 1]))
c = a + b
draw_graph(c, "demo_graph.typ")

**Question 2**:

After solving the previous question, your `BearTensor` should be able to construct the computation graph from the forward pass. In this part, we will implement the backwards pass, using the `BearGrad` functionality from above to propagate the gradients downstream.

**Conceptual Overview**

When training networks, we want to backpropagate the gradients starting from the very final node, which produced our training loss. To do this efficiently, we adopt a recursive approach; the loss node propagates its gradient back to its parent nodes. Then, every parent node propagates its gradient one layer further back; we keep recursively doing this until gradients have been propagated across the whole computation graph.

In lecture, we implemented single-variable `autograd` with two passes when `.backward()` is called: 
a "reset" pass that sets up counters for each node in our computational graph, 
and then a recursive backward pass to iterate through the entire graph and accumulate each node's adjoint.

We can actually perform both of these steps at once using an algorithm called <i>topological sort</i>.
This addresses the core problem we are trying to solve: a node can only propagate its gradient backward after it has received gradients from all its children.

[Topological sort](https://en.wikipedia.org/wiki/Topological_sorting) starts from the root node (where we call
`.backward()`), and traverses our computational graph to determine a correct order of operations. From Wikipedia: 
it provides a linear ordering that ensures that for any edge $u \to v$ in our computational graph, $u$ comes before
$v$ in the ordering. It turns out that *depth-first search (DFS)* actually gives us a topological sort!

You should first implement topological sort, then backprop; this ensures that for deep networks we do not encounter a stack overflow error. One implementation of this is [Kahn's algorithm](https://www.geeksforgeeks.org/dsa/topological-sorting-indegree-based-solution/) which is iterative instead of recursive, avoiding stack overflow errors for deep networks. Alternatively, you can use the implementation discussed in lecture.

**Your Task:**

Fill out `topological_sort` first, then `reset_children`, and `backward` below to implement backpropagation. After completing this, you finally have your very own PyTorch implementation!

In [ ]:
def topological_sort(node):
    '''Return a list of nodes ordered topologically'''
    ...
    return sorted_nodes

In [ ]:
def reset_children(self):
    """Resets the gradient in the current node to zero and all nodes before it in the computation graph."""
    ...
    pass

def backward(self):
    """
    Take a node in the computation graph, reset all gradients, and perform backpropagation 
    to compute the adjoints (gradients) for all nodes in the graph.

    Hint: After resetting the gradients, what should the gradient at the current node be?
    """
    ...
    pass

BearTensor.reset_children = reset_children
BearTensor.backward = backward

In [ ]:
grader.check("q2")

**Question 3: Optimizers**

Now that we are able to compute the gradients for all the parameters in our model, we want to be able to optimize them. In lecture we talked about Gradient Descent; in this homework, we will implement two widely-used optimizers, Stochastic Gradient Descent and Adam. It is highly recommended that you complete the paper/written portion of the homework before attempting this question.

**Your Task**:

Implement the SGD, Momentum, and Adam optimizers.

In [ ]:
class Optimizer:
    def __init__(self, params: list[BearTensor], lr: float):
        self.params = params
        self.lr = lr

    def zero_grad(self):
        for p in self.params:
            p.adjoint = np.zeros_like(p.value)

    def step(self):
        raise NotImplementedError

class SGD(Optimizer):
    def __init__(self, params: list[BearTensor], lr: float):
        super().__init__(params, lr)

    def step(self):
        ...
        pass

class Momentum(Optimizer):
    def __init__(self, params: list[BearTensor], lr: float, beta: float = 0.9):
        super().__init__(params, lr)
        self.beta = beta
        self.velocities = [np.zeros_like(p.value) for p in self.params]

    def step(self):
        ...
        pass


class Adam(Optimizer):
    def __init__(
        self,
        params: list[BearTensor],
        lr: float,
        beta1: float = 0.9,
        beta2: float = 0.999,
        eps: float = 1e-8,
    ):
        super().__init__(params, lr)
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self.t = 0
        self.ms = [np.zeros_like(p.value) for p in self.params]
        self.vs = [np.zeros_like(p.value) for p in self.params]

    def step(self):
        ...
        pass

In [ ]:
grader.check("q3")

The code below compares the convergence rates of all 3 optimizers. Play around with the learning rates and other hyperparameters to see how the results change.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

def compare_optimizers(optimizers):
    np.random.seed(0)
    X = BearTensor("X", np.random.randn(1000, 24))
    y = BearTensor("y", np.random.randn(1000, 1))
    
    n_steps = 100
    results = {}

    W1_init = np.random.randn(24, 12)
    W2_init = np.random.randn(12, 1)

    for opt_class, title, lr in optimizers:
        W1 = BearTensor("W1", deepcopy(W1_init))
        W2 = BearTensor("W2", deepcopy(W2_init))
        params = [W1, W2]

        optimizer = opt_class(params, lr=lr)
        losses = []

        for step in range(n_steps):
            optimizer.zero_grad()

            hidden = (X @ W1).sigmoid()
            output = hidden @ W2
            loss = ((output - y) ** 2).mean()

            loss.backward()
            losses.append(loss.value.item())
            optimizer.step()

        results[title] = losses

    plt.figure(figsize=(8, 5))
    for title, losses in results.items():
        plt.plot(losses, label=title, linewidth=2)
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Optimizer Convergence Comparison (Same Init)")
    plt.legend()
    plt.grid(True)
    plt.show()

compare_optimizers([
    (SGD, "SGD", 0.05),
    (Momentum, "Momentum", 0.05),
    (Adam, "Adam", 0.05)
])

**Question 4: Train a model!**

In this section, we're going to use the backpropagation engine and optimizers you created to train a model to predict wine quality scores!

**Conceptual Overview:**

Unfortunately, training our model is going to be slightly more involved than just using PyTorch. Our simple implementation does not allow us to define layers. Instead, we are going to define `BearTensors`, combine them manually to do a forward pass (which will automatically construct a computation graph!) and then finally backpropagate on our loss.

We are going to use the Wine Quality dataset, which contains 1599 samples of red wine. Each sample has 11 features representing various chemical properties, and the task is to predict the wine quality score (between 0 and 10). You can choose any loss function and optimizer (as long as you implemented it!) of your choice, feel free to play around with hyperparameters and different model sizes!

**Your Tasks:**

Train a model to predict wine quality scores. Your model should:
- Use at least one hidden layer
- Use an activation function for hidden layers
- Achieve a Mean Squared Error (MSE) of ≤ 2.0 on the dataset

**You need to fill in the `predict()` function that uses your architecture to return a prediction given an input datapoint. The autograder will use this to test your implementation**.

**Note**: If your model takes more than 30 seconds to train it could crash the autograder; you should easily be able to **achieve a MSE of ≤2** on the entire dataset with a 1-layer neural net for full credit. The staff solution takes <1 second to train, so if your code is slow, you may be constructing your graph inefficiently.

**You will receive 50% credit for this question if your MSE is ≤ 3.0 on the dataset.**

Tips for training models:
- Output your training and test loss every epoch; are they smoothly going down? If not, you may need to change the learning rate
- Ensure your computation graph is being built properly (remember our visualization helper functions from earlier!)
- Changing the number of layers or adding/removing bias terms can be a useful way to control how much your model is overfitting/underfitting

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

# DO NOT CHANGE THE PREPROCESSING CODE
data = fetch_openml("wine-quality-red", as_frame=True)
X = data.data.to_numpy()
y = data.target.to_numpy().astype(float)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.4, random_state=42, shuffle=True
)

X_tensor = BearTensor("X_train", X_train)               # shape: (N_train, input_dim)
y_tensor = BearTensor("y_train", y_train.reshape(-1,1)) # shape: (N_train,1)

# Add your training code here!
...

# --- Prediction function ---
def predict(x):
    """Output the scalar prediction for a single training datapoint x"""
    ...
    pass


## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
## Use this cell if you are running the notebook in Google Colab to install the necessary dependencies, this may take a few minutes
if IS_COLAB:
    !apt-get install -y texlive texlive-xetex pandoc


In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)